In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

from IPython.display import clear_output
from datetime import datetime

In [2]:
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter

wb = load_workbook('./crawler.xlsx')
ws = wb.active

for column in range(1, ws.max_column + 1):
    if ws.cell(1, column).value == 'Json_Src':
        column_src = column
    elif ws.cell(1, column).value == 'Picture':
        column_pic = column

print(f'总数量：{ws.max_row - 1}')

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(2, ws.max_row + 1):
    work.put_nowait(a)

# = = = = = = = = = = = = = = =

def crawler():
    global ws

    while not work.empty():
        a = work.get_nowait()

        # = = = = = = = = = = = = = = =

        crawler_status = 'error'

        # = = = = = = = = = = = = = = =

        import json

        src = json.loads(ws.cell(a, column_src).value).get('0', '')

        # = = = = = = = = = = = = = = =

        if src == '':
            b = 0
            crawler_status = 'ok'
        else:
            import requests

            import sys
            sys.path.append('../../00.Tools')
            from crawler_configuration import get_header, get_proxy

            try:
                b = 0
                while True:

                    while True:
                        b += 1
                        try:
                            resp = requests.get(src,
                                                headers=get_header(),
                                                proxies=get_proxy(),
                                                timeout=(10, 10))
                            break
                        except:
                            continue

                    if resp.status_code == 200:
                        break

                # = = = = = = = = = = = = = = =

                from io import BytesIO
                from openpyxl.drawing.image import Image

                pic = Image(BytesIO(resp.content))

                # = = = = = = = = = = = = = = =

                ws.add_image(pic, f'{get_column_letter(column_pic)}{a}')

                # = = = = = = = = = = = = = = =

                crawler_status = 'ok'
            except:
                pass

        # = = = = = = = = = = = = = = =

        if work.qsize() % 1_000 == 0 and work.qsize() != 0:
            clear_output()

        print(f'''[状态：{crawler_status}，尝试次数：{b}] - {src}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(12):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
wb.save(f'''./pic-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''')
print('Done ~')

[状态：ok，尝试次数：2] - https://cdn11.bigcommerce.com/s-z9vhe3o238/images/stencil/1280x1280/products/74890/5431245/482__23625.1687939757.jpg?c=1
[剩余数量：1000] - [当前时间：17:36:56]

[状态：ok，尝试次数：1] - https://cdn11.bigcommerce.com/s-z9vhe3o238/images/stencil/1280x1280/products/75131/5365081/482__77012.1687904083.jpg?c=1
[剩余数量：999] - [当前时间：17:36:58]

[状态：ok，尝试次数：1] - https://cdn11.bigcommerce.com/s-z9vhe3o238/images/stencil/1280x1280/products/91701/5386521/482__06621.1687916002.jpg?c=1
[剩余数量：998] - [当前时间：17:37:01]

[状态：ok，尝试次数：1] - https://cdn11.bigcommerce.com/s-z9vhe3o238/images/stencil/1280x1280/products/68174/5452884/482__37040.1687951312.jpg?c=1
[剩余数量：997] - [当前时间：17:37:03]

[状态：ok，尝试次数：1] - https://cdn11.bigcommerce.com/s-z9vhe3o238/images/stencil/1280x1280/products/57517/5424459/482__82787.1687936261.jpg?c=1
[剩余数量：996] - [当前时间：17:37:05]

[状态：ok，尝试次数：1] - https://cdn11.bigcommerce.com/s-z9vhe3o238/images/stencil/1280x1280/products/80635/5342095/482__87615.1687891852.jpg?c=1
[剩余数量：995] - [当前时间：17